# Age and Gender Recognition - Training the model

In this notebook we will train the model and save it into a file so we can use it later to detect the age and gender. Also, we will be running this notebook in Google Colab because we can use the GPU available.

In [4]:
# Imports
import cv2 as cv
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import keras.applications
import numpy as np
import pathlib
import gdown
#import os.path
import tarfile
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# Semilla aleatoria
keras.backend.clear_session()
np.random.seed = 42
tf.random.set_seed(42)

### **0. Preparing the dataset**

First of all we will download the dataset from the UTKFace url.

In [3]:
# We select the url
url = "https://drive.google.com/drive/folders/0BxYys69jI14kU0I1YUQyY1ZDRUE?usp=sharing"

# Then download the drive folder into 'face_dataset', if not exists
if os.path.isdir('./face_dataset/UTKFace/') == False:

  gdown.download_folder(url, output='face_dataset', quiet=True, use_cookies=False)

  # Now we will unzip the file
  file = tarfile.open('./face_dataset/UTKFace.tar.gz')

  # Extract the file
  file.extractall('./face_dataset')

  # Close the file
  file.close()

After downloading the data, we will transform the images into a Tensorflow dataset.

In [5]:
# We declare the batch size
batch_size = 32

path = 'face_dataset/UTKFace/'
ds_faces = tf.data.Dataset.list_files(str(pathlib.Path(path+'*.jpg')))

def process_path(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    label = tf.strings.split(file_path, '/')[2]
    label = tf.strings.split(label, '_')[0]# First test: only Age ":2]"
    label = tf.strings.to_number(label, out_type=tf.int64)

    return image, label

ds_faces = ds_faces.map(process_path).batch(batch_size)

Finally, we will divide the dataset between training, validation and test.

In [6]:
# First of all we will shuffle the data
ds_faces = ds_faces.shuffle(1000)

# Then we will split
train_size = 0.8
val_size = 0.1
test_size = 0.1
ds_len = ds_faces.__len__().numpy()

ds_train = ds_faces.take(int(train_size * ds_len))
ds_val = ds_faces.skip(int(train_size * ds_len)).take(int(val_size * ds_len))
ds_test = ds_faces.skip(int(train_size * ds_len + val_size * ds_len)).take(int(test_size * ds_len))

### **1. Data Augmentation**

Now, we will make data augmentation and data transformation. We will create transformation layers for the data augmentation and rescale/resize.

In [15]:
# We declare the sizes for the data augmentation
img_height = 140
img_width = 140

## Reescale
# Creating the reescaling layer
resize_rescale = keras.Sequential([
  keras.layers.Resizing(img_height, img_width),
  keras.layers.Rescaling(scale=1./127.5, offset=-1)
])

## Data Augmentation
# Rotate and flip
rotation_flip = keras.Sequential([
  keras.layers.RandomFlip("horizontal_and_vertical"),
  keras.layers.RandomRotation(0.2)
])

## Finally, we create a layer with the transformations
data_augmentation = keras.Sequential(
    [rotation_flip]
)

Now we will apply the layers to the dataset.

In [17]:
# Reescaling the images
ds_train = ds_train.map(lambda x, y: (tf.image.resize(x, (img_height, img_width)), y))
ds_val = ds_val.map(lambda x, y: (tf.image.resize(x, (img_height, img_width)), y))
ds_test = ds_test.map(lambda x, y: (tf.image.resize(x, (img_height, img_width)), y))

# We convert the data into batcher
ds_train = ds_train.cache().batch(batch_size).prefetch(buffer_size=10)
ds_val = ds_val.cache().batch(batch_size).prefetch(buffer_size=10)
ds_test = ds_test.cache().batch(batch_size).prefetch(buffer_size=10)

### **2. Configuring the pre-trained model**

In this step, we will select a pre-trained model (Xception)

In [7]:
model = keras.Sequential([
    layers.Input((200,200,3)),
    layers.Conv2D(16, 3, padding='same'),
    layers.Conv2D(32,3, padding='same'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(10)
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=[
        keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    ],
    metrics=['accuracy']
    )

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 200, 200, 32)      4640      
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 32)      0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 320000)            0         
                                                                 
 dense (Dense)               (None, 10)                3200010   
                                                                 
Total params: 3205098 (12.23 MB)
Trainable params: 3205098 (12.23 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [9]:
model.fit(ds_faces, epochs=10, verbose=2)

Epoch 1/10


KeyboardInterrupt: ignored